In [1]:
!pip install cassandra-driver

In [47]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2 , CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider

In [50]:
cluster_cassandra = Cluster(['vmcassandra.eastus2.cloudapp.azure.com'], port=9042)

In [51]:
session_cassandra = cluster_cassandra.connect()

In [52]:
command = """
CREATE KEYSPACE IF NOT EXISTS source

WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 2};
"""

In [6]:
session_cassandra.execute(command)

In [7]:
command = """
CREATE TABLE IF NOT EXISTS source.employees (
   no int PRIMARY KEY,
   name text)
"""

In [8]:
session_cassandra.execute(command)

In [10]:
command = """
USE source
"""

In [11]:
session_cassandra.execute(command)

In [14]:
command = """
insert into source.employees (no, name) VALUES(1, 'Varun')
"""

In [15]:
session_cassandra.execute(command)

In [18]:
!pip install pandas
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 KB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 68.9 MB/s eta 0:00:0000:0100:01


In [25]:
dfemployees = pd.read_csv("employees.csv")

In [26]:
dfemployees

,no,name
0,1,Atin
1,2,Chetan
2,3,Badri
3,4,Satish
4,5,Suresh
5,6,Sandeep
6,7,Manish
7,8,Parul
8,9,Pooja
9,10,Jaynesh


In [41]:
for emp in dfemployees.iterrows():
    #print(emp[0])
    emp_no = emp[1][0]
    emp_name = emp[1][1]
    #print(emp_no, emp_name)
    #print("--------------")
    command = f"""
    insert into source.employees (no, name) VALUES({emp_no}, '{emp_name}')
    """
    session_cassandra.execute(command)
    
    
    

/tmp/ipykernel_2807/2331190616.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  emp_no = emp[1][0]
/tmp/ipykernel_2807/2331190616.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  emp_name = emp[1][1]


In [54]:
pip install aws

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 KB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 KB 27.8 MB/s eta 0:00:00
Using legacy 'setup.py install' for aws, since package 'wheel' is not installed.
  Running setup.py install for aws ... done
Note: you may need to restart the kernel to use updated packages.


In [55]:
ssl_context = SSLContext(PROTOCOL_TLSv1_2 )
ssl_context.load_verify_locations('sf-class2-root.crt')

/tmp/ipykernel_2807/972152945.py:1: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2 )


In [56]:
ssl_context.verify_mode = CERT_REQUIRED
auth_provider = PlainTextAuthProvider(username='u1-at-044323040668', password='2/XrmsY7nIBQJCl0KBgZjhWY9P/SYM3DRPZ4w8dopRI=')

In [58]:
cluster_aws = Cluster(['cassandra.us-east-1.amazonaws.com'], ssl_context=ssl_context, auth_provider=auth_provider, port=9142)

In [59]:
session_aws = cluster_aws.connect()

In [83]:
command = """
select * from source.employees
"""

In [84]:
rows_source_emp = session_cassandra.execute(command)

In [85]:
command = """
CREATE KEYSPACE IF NOT EXISTS target

WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 2};
"""

In [86]:
session_aws.execute(command)

In [87]:
command = """
CREATE TABLE IF NOT EXISTS target.employees (
   no int PRIMARY KEY,
   name text)
"""

In [88]:
session_aws.execute(command)

In [91]:
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement

for row in rows_source_emp:
    query = SimpleStatement(
        "INSERT INTO  target.employees (no, name) VALUES (%s, %s)",
        consistency_level=ConsistencyLevel.LOCAL_QUORUM)
    session_aws.execute(query, row)

In [93]:
rows = session_aws.execute("select * from target.employees")

In [95]:
for row in rows:
    print(row)

In [ ]:
pd.read_json("employees.json")

In [96]:
! pip install amazon-dax-client
! pip install boto3

In [97]:
import boto3

In [98]:
dyn_resource = boto3.resource('dynamodb', region_name="us-east-1")

In [109]:
table_name = 'employees'
params = {
    'TableName': table_name,
    'KeySchema': [
        {'AttributeName': 'no', 'KeyType': 'HASH'},
        {'AttributeName': 'name', 'KeyType': 'RANGE'}
    ],
    'AttributeDefinitions': [
        {'AttributeName': 'no', 'AttributeType': 'N'},
        {'AttributeName': 'name', 'AttributeType': 'S'}
    ],
    'ProvisionedThroughput': {
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
}

In [110]:
table = dyn_resource.create_table(**params)

In [114]:
table = dyn_resource.create_table(**params)
print(f"Creating {table_name}...")
table.wait_until_exists()

ResourceInUseException: An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: employees

In [115]:
table_emp = dyn_resource.Table("employees")

In [116]:
table_emp.put_item(Item={
    'no': 1,
    'name': "Atin",
    'age': 40
}
)

{'ResponseMetadata': {'RequestId': 'IGP89B81EOAR9PCSG9I6416I8JVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 11 Oct 2023 11:15:37 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'IGP89B81EOAR9PCSG9I6416I8JVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [122]:
dfemployees

,no,name
0,1,Atin
1,2,Chetan
2,3,Badri
3,4,Satish
4,5,Suresh
5,6,Sandeep
6,7,Manish
7,8,Parul
8,9,Pooja
9,10,Jaynesh


In [129]:
ktr = 100
for emp in dfemployees.iterrows():
    print("ktr: ", ktr)
    print(emp[1][0], emp[1][1])
    ktr = ktr + 1

ktr:  100
1 Atin
ktr:  101
2 Chetan
ktr:  102
3 Badri
ktr:  103
4 Satish
ktr:  104
5 Suresh
ktr:  105
6 Sandeep
ktr:  106
7 Manish
ktr:  107
8 Parul
ktr:  108
9 Pooja
ktr:  109
10 Jaynesh


/tmp/ipykernel_2807/2995004661.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(emp[1][0], emp[1][1])


In [117]:
for emp in dfemployees.iterrows():
    emp_no = emp[1][0]
    emp_name = emp[1][1]
    obj_emp = {
            'no': emp_no,
            'name': emp_name,
    }
    obj_emp['address'] = "Noida"
    
    table_emp.put_item(Item=obj_emp)

/tmp/ipykernel_2807/1091565722.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  emp_no = emp[1][0]
/tmp/ipykernel_2807/1091565722.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  emp_name = emp[1][1]


In [155]:
command = """
select * from source.employees
"""
rows_source_emp = session_cassandra.execute(command)

In [156]:
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement

with table_emp.batch_writer() as batch:
    for row in rows_source_emp:
        obj_emp = {
                'no': row.no+100,
                'name': row.name + "-",
        }
        obj_emp['address'] = "From-Cassandra"
    
        batch.put_item(Item=obj_emp)

In [138]:
dict_employees = table_emp.scan()

In [151]:
for val in dict_employees['Items']:
    try:
        print(int(str(val['name'])))
    except:
        print("Some error occured")
        print(val['name'])

Some error occured
Manish
Some error occured
Jaynesh-
Some error occured
Parul
Some error occured
Satish-
Some error occured
Jaynesh
Some error occured
Badri
Some error occured
Chetan
Some error occured
Parul-
Some error occured
Pooja
Some error occured
Satish
Some error occured
Sandeep
Some error occured
Chetan-
Some error occured
Badri-
Some error occured
Sandeep-
Some error occured
Atin
Some error occured
Suresh-
Some error occured
Atin-
Some error occured
Pooja-
Some error occured
Suresh
Some error occured
Manish-


In [153]:
help(table_emp.batch_writer)

Help on method batch_writer in module boto3.dynamodb.table:

batch_writer(overwrite_by_pkeys=None) method of boto3.resources.factory.dynamodb.Table instance
    Create a batch writer object.
    
    This method creates a context manager for writing
    objects to Amazon DynamoDB in batch.
    
    The batch writer will automatically handle buffering and sending items
    in batches.  In addition, the batch writer will also automatically
    handle any unprocessed items and resend them as needed.  All you need
    to do is call ``put_item`` for any items you want to add, and
    ``delete_item`` for any items you want to delete.
    
    Example usage::
    
        with table.batch_writer() as batch:
            for _ in range(1000000):
                batch.put_item(Item={'HashKey': '...',
                                     'Otherstuff': '...'})
            # You can also delete_items in a batch.
            batch.delete_item(Key={'HashKey': 'SomeHashKey'})
    
    :type overwrite_

In [1]:
! python3 -m pip install couchbase

In [5]:
from datetime import timedelta

In [1]:
# needed for any cluster connection
from couchbase.auth import PasswordAuthenticator

In [2]:
from couchbase.cluster import Cluster
# needed for options -- cluster, timeout, SQL++ (N1QL) query, etc.
from couchbase.options import (ClusterOptions, ClusterTimeoutOptions,
                               QueryOptions)

ImportError: cannot import name 'ClusterOptions' from 'couchbase.options' (/pyenv/lib/python3.10/site-packages/couchbase/options.py)